# KoBERT

- SKT T-Brain에서 open source로 제공하는 한국어 BERT <br>
https://github.com/SKTBrain/KoBERT
- Huggingface의 monologg/kobert 참고 <br>
https://github.com/monologg/KoBERT-Transformers

- 안정적으로 활용할 수 있는 검증된 라이브러리 버전 <br>
```
torch >= 1.1.0
mxnet >= 1.4.0
gluonnlp >= 0.6.0
sentencepiece >= 0.1.6
onnxruntime >= 0.3.0
transformers >= 2.1.1, <= 3.0.2
```

In [1]:
!pip install gluonnlp
!pip install mxnet

     |████████████████████████████████| 348kB 3.4MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588519 sha256=81bd153258a15550a7a8dff587b2acfacbbc324728c5d92bb672e8e50b48d100
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 55.0MB 82kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [2]:
import os
import sys
import requests
import hashlib

In [4]:
!pip install transformers==3.0.2

     |████████████████████████████████| 778kB 3.3MB/s 
     |████████████████████████████████| 3.0MB 11.2MB/s 
     |████████████████████████████████| 890kB 26.0MB/s 
     |████████████████████████████████| 1.1MB 33.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4386ca1a09b1a2d14255ae126c513fd97df6314bf0c40ca2fcdc1d21cf6fde99
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import torch
from transformers import BertModel, BertConfig
import gluonnlp as nlp

In [6]:
onnx_kobert = {
    'url':
    'https://kobert.blob.core.windows.net/models/kobert/onnx/onnx_kobert_44529811f0.onnx',
    'fname': 'onnx_kobert_44529811f0.onnx',
    'chksum': '44529811f0'
}

tokenizer = {
    'url':
    'https://kobert.blob.core.windows.net/models/kobert/tokenizer/kobert_news_wiki_ko_cased-ae5711deb3.spiece',
    'fname': 'kobert_news_wiki_ko_cased-1087f8699e.spiece',
    'chksum': 'ae5711deb3'
}

In [7]:
def download(url, filename, chksum, cachedir='kobert/'):
    f_cachedir = os.path.expanduser(cachedir)
    os.makedirs(f_cachedir, exist_ok=True)
    file_path = os.path.join(f_cachedir, filename)
    if os.path.isfile(file_path):
        if hashlib.md5(open(file_path,
                            'rb').read()).hexdigest()[:10] == chksum:
            print('using cached model')
            return file_path
    with open(file_path, 'wb') as f:
        response = requests.get(url, stream=True)
        total = response.headers.get('content-length')

        if total is None:
            f.write(response.content)
        else:
            downloaded = 0
            total = int(total)
            for data in response.iter_content(
                    chunk_size=max(int(total / 1000), 1024 * 1024)):
                downloaded += len(data)
                f.write(data)
                done = int(50 * downloaded / total)
                sys.stdout.write('\r[{}{}]'.format('█' * done,
                                                   '.' * (50 - done)))
                sys.stdout.flush()
    sys.stdout.write('\n')
    assert chksum == hashlib.md5(open(
        file_path, 'rb').read()).hexdigest()[:10], 'corrupted file!'
    return file_path

In [8]:
def get_onnx(cachedir='kobert/'):
    """Get KoBERT ONNX file path after downloading
    """
    model_info = onnx_kobert
    return download(model_info['url'],
                    model_info['fname'],
                    model_info['chksum'],
                    cachedir=cachedir)

In [9]:
def get_tokenizer(cachedir='kobert/'):
    """Get KoBERT Tokenizer file path after downloading
    """
    model_info = tokenizer
    return download(model_info['url'],
                    model_info['fname'],
                    model_info['chksum'],
                    cachedir=cachedir)

In [10]:
pytorch_kobert = {
    'url':
    'https://kobert.blob.core.windows.net/models/kobert/pytorch/pytorch_kobert_2439f391a6.params',
    'fname': 'pytorch_kobert_2439f391a6.params',
    'chksum': '2439f391a6'
}

bert_config = {
    'attention_probs_dropout_prob': 0.1,
    'hidden_act': 'gelu',
    'hidden_dropout_prob': 0.1,
    'hidden_size': 768,
    'initializer_range': 0.02,
    'intermediate_size': 3072,
    'max_position_embeddings': 512,
    'num_attention_heads': 12,
    'num_hidden_layers': 12,
    'type_vocab_size': 2,
    'vocab_size': 8002
}

In [11]:
def get_pytorch_kobert_model(ctx='cpu', cachedir='kobert/'):
    # download model
    model_info = pytorch_kobert
    model_path = download(model_info['url'],
                           model_info['fname'],
                           model_info['chksum'],
                           cachedir=cachedir)
    # download vocab
    vocab_info = tokenizer
    vocab_path = download(vocab_info['url'],
                           vocab_info['fname'],
                           vocab_info['chksum'],
                           cachedir=cachedir)
    return get_kobert_model(model_path, vocab_path, ctx)

In [13]:
def get_kobert_model(model_file, vocab_file, ctx="cpu"):
    bertmodel = BertModel(config=BertConfig.from_dict(bert_config))
    bertmodel.load_state_dict(torch.load(model_file))
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [14]:
class KoBERTEmbedding:
    '''
    Extract KoBERT embedding from the last hidden layer
    '''
    def __init__(self):
        self.bert, self.vocab, self.tokenizer, self.tok = self.retrieve_kobert()

    def retrieve_kobert(self):
        bert, vocab = get_pytorch_kobert_model()
        tokenizer = get_tokenizer()
        tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

        return bert, vocab, tokenizer, tok

  # 안녕 하 세요
    def predict(self, sent):
        self.bert.eval()
        tokenized_text = self.tok(sent)
        indexed_tokens = self.tok.convert_tokens_to_ids(tokenized_text)
        segments_ids = [1] * len(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        with torch.no_grad():
            encoded_layers, _ = self.bert(tokens_tensor, segments_tensors)

        # token_vecs = encoded_layers[0]
        sentence_embedding = torch.mean(encoded_layers[0],dim=0)

        return sentence_embedding


In [15]:
bert_embedding = KoBERTEmbedding()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
using cached model


In [16]:
sent = '궁금한게 있어서 전화 문의드립니다.'
bert_embedding_vec = bert_embedding.predict(sent)

In [ ]:
print(len(bert_embedding_vec))
print(bert_embedding_vec.detach().numpy())

In [ ]:
!pip list